In [ ]:
# Before running: pip install numpy sklearn torchvision comet_ml
from comet_ml import Experiment  # has to be 1st import
import numpy as np
import sklearn
from sklearn.linear_model import Perceptron
from torchvision import datasets, transforms

In [ ]:
# INSERT YOUR DATA HERE
train_data = "path/to/data/dir"  # required
val_data = None                  # optional
test_data = None                 # optional

In [ ]:
# Set up globals.
model = Perceptron()
experiment = Experiment("hellsoiower")

In [ ]:
def preprocess(data, name):
    if data is None:  # val/test can be empty
        return None

    # Read image files to pytorch dataset (only temporary).
    transform = transforms.Compose(
        [transforms.Resize(28), transforms.CenterCrop(26), transforms.ToTensor()]
    )
    data = datasets.ImageFolder(data, transform=transform)
    
    # Convert to numpy arrays.
    images_shape = (len(data), *data[0][0].shape)
    images = np.zeros(images_shape)
    labels = np.zeros(len(data))
    for i, (image, label) in enumerate(data):
        images[i] = image
        labels[i] = label

    # Flatten.
    images = images.reshape(len(images), -1)

    # Shuffle train set.
    if name == "train":
        images, labels = sklearn.utils.shuffle(images, labels)

    return [images, labels]

In [ ]:
def evaluate(data, name):
    if data is None:  # val/test can be empty
        return

    images, labels = data
    acc = model.score(images, labels)
    print(f"{name} accuracy:".ljust(18) + str(acc))
    experiment.log_metric(f"{name}_accuracy", acc)

In [ ]:
def train(train_data, val_data=None, test_data=None):
    # Train on train_data.
    train_images, train_labels = train_data
    model.fit(train_images, train_labels)

    # Evaluate on all datasets.
    evaluate(train_data, "train")
    evaluate(val_data, "val")
    evaluate(test_data, "test")

In [ ]:
train_data = preprocess(train_data, "train")
val_data = preprocess(val_data, "val")
test_data = preprocess(test_data, "test")

In [ ]:
train(train_data, val_data, test_data)